# Control- and Data- Flow Graph Drawer
## References:
- akashlevy/LLVM-Dataflow-Examples (https://github.com/akashlevy/LLVM-Dataflow-Examples)
- llvmlite/examples/notebooks (https://github.com/numba/llvmlite/blob/master/examples/notebooks/Visualize%20ControlFlow.ipynb)

## Prerequisites
- llvmlite
- graphviz

## License
- Apache v2.0

In [2]:
from llvmlite import ir
from llvmlite import binding as llvm
from llvmlite import llvmpy

Build a simple function with a for loop that sum the first N integer, where N is the argument.

In [ ]:
m = ir.Module()
fnty = ir.FunctionType(ir.IntType(32), [ir.IntType(32)])
fn = ir.Function(m, fnty, "count_number")
fn.args[0].name= 'N'
builder = ir.IRBuilder(fn.append_basic_block('entry'))
out = builder.alloca(ir.IntType(32), name='out')
ct = builder.alloca(ir.IntType(32), name='ct')
builder.store(out.type.pointee(0), out)
builder.store(ct.type.pointee(0), ct)
loophead = fn.append_basic_block('loop.header')
loopbody = fn.append_basic_block('loop.body')
loopend = fn.append_basic_block('loop.end')

builder.branch(loophead)
builder.position_at_end(loophead)

# loop if ct < arg0
arg0 = fn.args[0]
pred = builder.icmp_signed('<', builder.load(ct), arg0)
builder.cbranch(pred, loopbody, loopend)

builder.position_at_end(loopbody)

# out += ct
builder.store(builder.add(builder.load(out), builder.load(ct)), out)
# ct += 1
builder.store(builder.add(builder.load(ct), ct.type.pointee(1)), ct)
# jump to loophead
builder.branch(loophead)

builder.position_at_end(loopend)

builder.ret(builder.load(out))

dot = llvm.get_function_cfg(fn)
llvm.view_dot_graph(dot)

# materialize a LLVM module
mod = llvm.parse_assembly(str(m))

# create optimizer
pm = llvm.create_module_pass_manager()
pmb = llvm.create_pass_manager_builder()
pmb.opt_level = 3  # -O3
pmb.populate(pm)

# optimize
pm.run(mod)

## Visualize CDFG

In [ ]:
class Graph:

    def __init__(self, f, out, options):
        self.f = f
        self.out = out
        self.options = options
        self.edges = []
        self.anon_bblock_cnt = 0
        self.anon_bblock_names = {}
        self.void_instr_cnt = 0
        self.void_instr_names = {}

    def write(self, line=""):
        self.out.write(line + "\n")

    def start_graph(self):
        self.write("digraph G {")
        self.write("compound=true")
        if self.options.dag_control:
            self.write("rankdir=BT")
        if self.options.block_edges and not self.options.block_edges_helpers:
            # If we use cluster edges w/o intervening nodes, we need to bump
            # rank (vertical) separation, because otherwise there's very
            # little vert. space left to render edges after cutting out
            # cluster rectangle
            self.write("ranksep=1")
        self.write('label="Black edges - dataflow, red edges - control flow"')

    def edge(self, fro, to, extra=""):
        self.edges.append("\"%s\" -> \"%s\"%s" % (fro, to, extra))

    def block_name(self, b):
        """Returns basic block name, i.e. its entry label, or made name
        if label if absent."""
        if b.name:
            return b.name
        if b in self.anon_bblock_names:
            return self.anon_bblock_names[b]
        self.anon_bblock_cnt += 1
        n = "unk_block_%d" % self.anon_bblock_cnt
        self.anon_bblock_names[b] = n
        return n

    def instr_name(self, i):
        """Returns instruction name, for which result variable name is used.
        If result variable name is absent (void statement), make up name.
        """
        if i in self.void_instr_names:
            return self.void_instr_names[i]
        n = i
        if not n:
            self.void_instr_cnt += 1
            n = "_%d" % self.void_instr_cnt
            self.void_instr_names[i] = n
        return n

    def declare_clusters(self):
        if self.options.block:
            for b in self.f.blocks:
                name = self.block_name(b)
#                if not self.options.block_edges_helpers:
                if 1:
                    self.write("subgraph \"cluster_%s\" {" % name)

                if not self.options.block_edges:
                    self.write('\"%s\" [label="label: \"%s\""]' % (name, name))
                elif self.options.block_edges_helpers:
                    self.write('\"%s\" [shape=point height=0.02 width=0.02 color=red fixedsize=true]' % name)

#                    if not self.options.block_edges_helpers:
                if 1:
                    self.write("}")
            self.write()


    def render(self):
        self.start_graph()
        self.declare_clusters()
        lab = 1
        for b in self.f.blocks:
            block_name = self.block_name(b)
            self.edges = []
            if self.options.block:
                self.write("subgraph \"cluster_%s\" {" % block_name)
                self.write("label=%s" % block_name)
#            if not self.options.block_edges:
#                self.write('\"%s\" [label="label: %s"]' % (block_name, block_name))
#           elif self.options.block_edges_helpers:
#               self.write('\"%s\" [shape=point]' % (b.name))

            # Create block entry label node and edge from it to first IR instruction
            if not self.options.block_edges or self.options.block_edges_helpers:
                attr = "[color=red]"
                if b.name == "entry":
                    attr += "[weight=5]"
                if self.options.block_edges:
                    attr += "[lhead=\"cluster_%s\"]" % block_name
                if self.options.control:
                    for index, i in enumerate(b.instructions):
                        if index == 0:
                            if self.instr_name(i) == "":
                                n = self.instr_name(i)
                                self.edge(block_name, n, attr)
                            else:
                                self.edge(block_name, self.instr_name(i), attr)

            if self.options.dag_control:
                last_void_inst = block_name
                for i in b.instructions:
                    if i.type == None:#:
                        n = self.instr_name(i)
                        self.edge(n, last_void_inst, "[color=blue dir=back]")
                        last_void_inst = n

            last_inst_name = None
            for i in b.instructions:
                n = self.instr_name(i)
                self.write('\"%s\" [label="%s"]' % (n, i))
                if self.options.control:
                    if last_inst_name:
                        self.edge(last_inst_name, n, "[color=red weight=2]")
                else:
                    if self.instr_name(i) == "br" and len(i.operands) == 1:
                        self.edge(last_inst_name, n, "[color=red]")

                for a in i.operands:
                        if not a.name:
                            arg_val = a
                        else:
                            arg_val = a.name
                        if self.instr_name(i) == "br" and type(a) is BasicBlock:
                            if self.options.block_edges and not self.options.block_edges_helpers:
                                arg_val = a.instructions[0].name
                            attrs = "[color=red]"
                            if self.options.block_edges:
                                attrs += "[color=red][lhead=\"cluster_%s\"][ltail=\"cluster_%s\"][weight=5]" % (a.name, block_name)
                                if self.options.block_edges_helpers:
                                    attrs += "[arrowhead=none]"
                            self.edge(n, arg_val, attrs)
                        else:
                            self.edge(arg_val, n)
                last_inst_name = n
            if self.options.block:
                self.write("}")
            for e in self.edges:
                self.write(e)
            self.write()
        self.write("}")

## Draw Function-level CFG

In [ ]:
dot = llvm.get_function_cfg(fn)
llvm.view_dot_graph(dot)

## Preparation to Visualization

In [ ]:
# materialize a LLVM module
mod = llvm.parse_assembly(str(m))

In [ ]:
llvm.initialize()
llvm.initialize_native_target()
llvm.initialize_native_asmprinter()

# materialize a LLVM module
mod = llvm.parse_assembly(str(m))

In [ ]:
# create optimizer
pm = llvm.create_module_pass_manager()
pmb = llvm.create_pass_manager_builder()
pmb.opt_level = 3  # -O3
pmb.populate(pm)

# optimize
pm.run(mod)

## Options for Visualization

In [ ]:
class options:
  block=False
  control=True
  dag_control=True
  block_edges=False
  block_edges_helpers=False

## Dot file Generation

In [ ]:
if not options.control and not options.dag_control:
    options.control = True
    
for f in mod.functions:
    if not f.is_declaration:
        print("Writing %s.dot" % f.name)
        with open(f.name + ".dot", "w") as out:
            g = Graph(f, out, options)
            g.render()


In [ ]:
!dot -Tpdf count_number.dot -o outfile.pdf

In [ ]:
import pydotplus
from IPython.display import Image

In [ ]:
graph = pydotplus.graphviz.graph_from_dot_file("count_number.dot")
graph.write_png("count_number.png")
Image(graph.create_png())

In [ ]:
dot = llvm.get_function_cfg(mod.get_function(fn.name))
llvm.view_dot_graph(dot)